In [ ]:
from tensorflow import keras
import shutil
import matplotlib.pyplot as plt
import cv2
from google.colab import drive 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import re
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dropout,Dense,BatchNormalization,Flatten
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
shutil.unpack_archive("/content/gdrive/My Drive/Leaf_Images.zip", "/content/")

In [ ]:
folders=os.listdir('/content/Leaf_Images/')
for i in folders:
  files=os.listdir('/content/Leaf_Images/'+i+'/output/')
  for j in files:
    os.rename('/content/Leaf_Images/' +i+ '/output/' +j, '/content/Leaf_Images/' +i+ '/' +j)
  os.rmdir('/content/Leaf_Images/' +i+ '/output/')

In [ ]:
for i in sorted(folders):
  if(i!='.ipynb_checkpoints'):
    if(int(i)>1 ):
      os.rename('/content/Leaf_Images/'+str(i),'/content/Leaf_Images/'+str(int(i)-1))

In [ ]:
folders=os.listdir('/content/Leaf_Images/')

In [ ]:
if(not os.path.exists('/content/Leaf_Images/train')):
  os.makedirs('/content/Leaf_Images/train')
if(not os.path.exists('/content/Leaf_Images/valid')):
  os.makedirs('/content/Leaf_Images/valid')
if(not os.path.exists('/content/Leaf_Images/test')):
  os.makedirs('/content/Leaf_Images/test')
for i in folders:
  if(i!='.ipynb_checkpoints'):
    if(not os.path.exists('/content/Leaf_Images/train/'+i)):
        os.makedirs('/content/Leaf_Images/train/'+i)
    if(not os.path.exists('/content/Leaf_Images/valid/'+i)):
        os.makedirs('/content/Leaf_Images/valid/'+i)
    if(not os.path.exists('/content/Leaf_Images/test/'+i)):
        os.makedirs('/content/Leaf_Images/test/'+i)

In [ ]:
for i in folders:
  files=os.listdir('/content/Leaf_Images/'+i)
  length_1=int(0.7*len(files))
  length_2=int(0.85*len(files)) 
  for j in files[:length_1]:
    if(j!='.ipynb_checkpoints' and i!='.ipynb_checkpoints'):
      os.rename('/content/Leaf_Images/' +i+ '/' +j,'/content/Leaf_Images/train/'+i+'/'+j)
  for j in files[length_1:length_2]:
    if(j!='.ipynb_checkpoints' and i!='.ipynb_checkpoints'):
      os.rename('/content/Leaf_Images/' +i+ '/' +j,'/content/Leaf_Images/valid/'+i+'/'+j)
  for j in files[length_2:]:
    if(j!='.ipynb_checkpoints' and i!='.ipynb_checkpoints'):
      os.rename('/content/Leaf_Images/' +i+ '/' +j,'/content/Leaf_Images/test/'+i+'/'+j)

In [ ]:
folders=os.listdir('/content/Leaf_Images/')
fdrs=['train','valid','test']
for i in folders:
  if(i not in fdrs):
    os.rmdir('/content/Leaf_Images/'+i)


In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode="nearest",
        brightness_range=[0.2,1.0],
        width_shift_range=0.2,
        height_shift_range=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        '/content/Leaf_Images/train',
        target_size=(120, 120),
        batch_size=32,
        class_mode='categorical')
valid_generator = test_datagen.flow_from_directory(
        '/content/Leaf_Images/valid',
        target_size=(120, 120),
        batch_size=32,
        class_mode='categorical')


Found 10303 images belonging to 7 classes.
Found 2209 images belonging to 7 classes.


In [ ]:
model=Sequential()
input_shape=(120,120,3)
model.add(Conv2D(32,(3,3),padding='same',activation='relu',input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))
model.add(Conv2D(64,(3,3),padding='same',activation='relu',input_shape=input_shape))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',activation='relu',input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))
model.add(Conv2D(128,(3,3),padding='same',activation='relu',input_shape=input_shape))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',activation='relu',input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(7,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 120, 120, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 120, 120, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 40, 40, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 40, 40, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 40, 40, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 40, 40, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 40, 40, 64)        3

In [ ]:
optimizer= tf.optimizers.Adam(lr=1e-3)
model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])
history=model.fit(train_generator,epochs=30, shuffle=True, validation_data=valid_generator)

Epoch 1/30
322/322 [==============================] - 474s 1s/step - loss: 2.0364 - accuracy: 0.3842 - val_loss: 4.7047 - val_accuracy: 0.3010
Epoch 2/30
322/322 [==============================] - 442s 1s/step - loss: 1.2796 - accuracy: 0.5706 - val_loss: 2.1585 - val_accuracy: 0.4572
Epoch 3/30
322/322 [==============================] - 438s 1s/step - loss: 1.0121 - accuracy: 0.6556 - val_loss: 1.0030 - val_accuracy: 0.7035
Epoch 4/30
322/322 [==============================] - 441s 1s/step - loss: 0.8393 - accuracy: 0.7005 - val_loss: 0.8437 - val_accuracy: 0.6931
Epoch 5/30
322/322 [==============================] - 440s 1s/step - loss: 0.7893 - accuracy: 0.7255 - val_loss: 1.1336 - val_accuracy: 0.7189
Epoch 6/30
322/322 [==============================] - 440s 1s/step - loss: 0.7448 - accuracy: 0.7459 - val_loss: 0.5547 - val_accuracy: 0.8112
Epoch 7/30
322/322 [==============================] - 440s 1s/step - loss: 0.6884 - accuracy: 0.7574 - val_loss: 0.8867 - val_accuracy: 0.7184

In [ ]:
model.save("model.h5")

Hyperparamter Tuning:

The CNN classifiers were built for bounding boxes with different sizes 120 x 120, 100 x 100 and 80 x 80

The learning rate was initially used was around the range 1e-3 and when the learning rate was switched to 3e-4, a learning rate which usually works best with Adam optimizer, there was a dip in accuracy by 10%.

This could be accounted to the fact that the slow learning rate takes longer to converge but is more probable to end up in a global minima. Owing to time and resource constraints, each model was trained only for 20-30 epochs. With increased epochs, a learning rate of 3e-4 would provide better accuracy.

Also, changing the depth of the network by a maximum of 2 layers showed no impact (showed just 1-2 % improvement in accuracy)

Changing the drop out values showed similar results.